In [1]:
# look at high-frequency words in top performers' resumes vs in bottom performers
# look at topics/words by PLSA in top and bottom bins and see overlaps 
import pandas as pd
import numpy as np

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

# Topic Modelling Analysis  
Looking at overlapping topics/words in the top and bottom employee bins. We want to suggest words to our client that are only in the top bin. Suggestions for key words to look for in applicants resume. 

In [2]:
#Create a pandas data frame 
data = pd.read_csv("resume_words.tsv",sep="\t")
type(data)
data.columns
len(data)

426

In [3]:
#What the data looks like 
data.head(5)


,emp_id,durationSinceFirstEngagement,performance_indicator,aa,aaa,aaliyah,aaron,aas,abandoned,abc,...,yrs,zachary,zeta,zion,zip,zone,zoo,zoological,zumba,zurich
0,C320K60000K0,315.0,0.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CLS2PN001040,80.0,0.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CLMXHR000040,86.0,0.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CNF9JR000040,49.0,0.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CLL1KT000040,83.0,0.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
from sklearn.feature_extraction.text import TfidfTransformer  
import nltk  
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vanessabayubaskoro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vanessabayubaskoro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# Tenure and Performance - numbers are from Jim
# Top Category (above these scores) 
good_per = 0.37
good_dur = 226
# Bottom Category (below these scores)
bad_per = 0.27
bad_dur = 84

In [29]:
# Grab Top Category employees index, tenure-based 

index_ls = []

for i in range(len(data)):
    if ( data.iloc[[i]]['durationSinceFirstEngagement'] > good_dur).bool():
        index_ls.append(i)
        
print("Top employees indeces based on tenure: " )
print(index_ls)
print("Number of top employees based on tenure: ")
print(len(index_ls))
        
# Grab Bottom Categorry employees index, tenure-based
bottom_index = []

for i in range(len(data)):
    if ( data.iloc[[i]]['durationSinceFirstEngagement'] < bad_dur).bool():
        bottom_index.append(i)
        
print("Bottom employees indeces based on tenure: ")
print(bottom_index)
print("Number of bottom employees based on tenure: ")
print(len(bottom_index))



Top employees indeces based on tenure: 
[0, 5, 8, 10, 15, 17, 44, 56, 71, 72, 73, 74, 75, 76, 77, 79, 80, 82, 83, 87, 89, 95, 101, 104, 112, 114, 121, 122, 127, 133, 138, 140, 145, 149, 163, 169, 172, 186, 187, 189, 190, 191, 193, 195, 197, 201, 202, 203, 208, 209, 210, 214, 216, 218, 220, 222, 223, 226, 229, 230, 231, 232, 233, 234, 236, 239, 240, 241, 242, 243, 246, 247, 249, 257, 260, 263, 269, 273, 277, 280, 283, 284, 287, 288, 289, 290, 292, 294, 296, 298, 299, 301, 302, 303, 305, 306, 316, 318, 322, 323, 324, 325, 326, 329, 330, 332, 334, 337, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 354, 356, 357, 358, 363, 365, 366, 368, 369, 373, 381, 385, 387, 392, 393, 394, 396, 397, 398, 403, 405, 407, 411, 412, 413, 417, 419, 421]
Number of top employees based on tenure: 
148
Bottom employees indeces based on tenure: 
[1, 3, 4, 11, 14, 16, 18, 22, 25, 26, 27, 28, 29, 31, 33, 45, 46, 48, 52, 53, 55, 58, 60, 63, 64, 67, 69, 70, 78, 86, 91, 92, 96, 97, 106, 111, 113, 115, 1

In [33]:
# Compile all words from a resume as one string for all employees in a category 
X = []
for i, (index,row) in enumerate(data.iterrows()):
    if i in index_ls: 
        docwords = data.iloc[i,3:]
        X.append( ' '.join(list(docwords[docwords==1].index.values) ) )
        
        
len(X) 


X2 = []
for i, (index,row) in enumerate(data.iterrows()):
    if i in bottom_index: 
        docwords = data.iloc[i,3:]
        X2.append( ' '.join(list(docwords[docwords==1].index.values) ) )
len(X2)

    
    

139

In [8]:
from nltk.stem import WordNetLemmatizer
import re

stemmer = WordNetLemmatizer()
documents = []



stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    ##Remove word numbers
    #document = re.sub(" \d+", " ", document)
    
    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
   # list_of_locations = ["chicago","minnesota","minneapolis", " dc ", " il ", " mn ", " wa ", "washington", "illinois", "maryland", " md "]

    #for city in list_of_locations:
     #   document = document.replace(city, " ")


    documents.append(document)

In [13]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


In [16]:
# Use tf-idf features for NMF.
n_samples = 2000
n_features = 1000
n_top_words = 20
n_components = 6

print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, 
                                   min_df=2,
                                   max_features=n_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(X)

Extracting tf-idf features for NMF...


## Top Tenure Category PLSA

In [17]:
# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))

nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)


print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=2000 and n_features=1000...

Topics in NMF model (generalized Kullback-Leibler divergence):
Topic #0: staff ability professional diploma objective services assistant summary set goals new calls communication according references safety knowledge management including excellent
Topic #1: english gpa communication degree year spanish community college social volunteer word society computer club events awards professional microsoft diploma bilingual
Topic #2: september associate october july responsible sales march additional employer april certifications ambassador authorized yahoo person college modern certified information state
Topic #3: diploma clean environment cleaning stocked area dining foods prep tasks duties community aid properly drinks new children use maintained preparing
Topic #4: college cashier restaurant organized ave diploma environment able self inventory fast quality motivat

## Bottom Tenure Category PLSA

In [34]:
tfidf2 = tfidf_vectorizer.fit_transform(X2)
nmf2 = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf2)
print_top_words(nmf2, tfidf_feature_names, n_top_words)

Topic #0: excel counting outreach offers va tickets language balanced correct desk establishment mcdonalds networking purchase classroom association coach updated honor associates
Topic #1: bathrooms leader consistently coordinator believe culinary workers online assisting correct guest presentations enthusiasm offers yahoo drink met level clean floor
Topic #2: multitask worked extensive administration prepping main court opportunity stay served grill mentor pulling monitoring fast condition stations profile company calm
Topic #3: store qualifications tickets drink flexible executed fall station mail consistently best dedicated paced having member pulling routine listening marlboro basset
Topic #4: opening receive registers benefits clean look drink reported entering hard complaints culinary trash awards locate changes completed associate association concerns
Topic #5: point cook awards oversaw consistently applied works years offers ability proficient establishment chicken graduated s